In [ ]:
# Force complete reimport of model module with all dependencies
import sys
import importlib

# Remove all model-related modules from cache
modules_to_remove = [key for key in sys.modules.keys() if 'model' in key.lower()]
for module_name in modules_to_remove:
    del sys.modules[module_name]

# Reimport fresh
import model
importlib.reload(model)
print("✓ Model module reloaded successfully")
print(f"✓ NoiseInjectionLayer available: {hasattr(model, 'NoiseInjectionLayer')}")
print(f"✓ PricePredictor available: {hasattr(model, 'PricePredictor')}")

In [ ]:
import os
import importlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, losses, initializers, regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, f1_score, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
import joblib
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FuncFormatter

# Reload project model module to pick up latest edits when iterating in the notebook
import model
importlib.reload(model)
from model import *
config = Config()

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import time
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import Button, Output, VBox, HBox, Label, FloatProgress
from contextlib import contextmanager

# Global variables for persistence
estimated_time_per_batch = None
total_time = None
pause_training = False
stop_training = False

class InteractivePlotCallback(tf.keras.callbacks.Callback):
    def __init__(self, config, loss_output, metrics_output, progress_widget, total_epochs, batch_output=None, batch_metrics_output=None):
        super().__init__()
        self.config = config
        self.loss_output = loss_output
        self.metrics_output = metrics_output
        self.progress_widget = progress_widget
        self.total_epochs = total_epochs
        self.history = {}
        self.epoch_count = 0

        # Batch-level widgets and history (text output no longer used)
        self.batch_metrics_output = batch_metrics_output or Output()
        self.batch_history = {}  # stores lists per metric for current epoch
        self.total_batches = None

    def on_epoch_begin(self, epoch, logs=None):
        """Flush batch-level displays and reset batch history at the start of each epoch."""
        # Reset per-epoch batch history
        self.batch_history.clear()
        self.batch_count = 0
        # total batches may be available in params
        self.total_batches = self.params.get('steps') if self.params is not None else None

        # Clear batch plot output
        with self.batch_metrics_output:
            clear_output(wait=True)

    def on_train_batch_end(self, batch, logs=None):
        """Capture per-batch logs and update batch plot only (no text logging)."""
        logs = logs or {}
        # Keras batch index starts at 0; store 1-based
        batch_idx = (batch or 0) + 1
        self.batch_history.setdefault('batch', []).append(batch_idx)
        for key, value in logs.items():
            try:
                self.batch_history.setdefault(key, []).append(float(value))
            except Exception:
                # skip non-numeric values
                pass
        self.batch_count = len(self.batch_history['batch'])

        # Update plot only
        try:
            self.update_batch_plot()
        except Exception:
            # Avoid breaking training if plotting fails
            pass

    def on_epoch_end(self, epoch, logs=None):
        global pause_training, stop_training
        if stop_training:
            self.model.stop_training = True
            return
        if pause_training:
            while pause_training and not stop_training:
                time.sleep(0.1)
            if stop_training:
                self.model.stop_training = True
                return

        logs = logs or {}
        self.epoch_count = epoch + 1

        # Collect all losses and metrics for epoch-level persistent history
        loss_types = ['loss', 'val_loss', 'point_loss', 'val_point_loss', 'trend_loss', 'val_trend_loss',
                      'local_trend_loss', 'val_local_trend_loss', 'global_trend_loss', 'val_global_trend_loss',
                      'extended_trend_loss', 'val_extended_trend_loss', 'dir_loss', 'val_dir_loss',
                      'reg_loss', 'val_reg_loss', 'vol_loss', 'val_vol_loss', 'var_nll', 'val_var_nll']
        for m in loss_types:
            if m in logs:
                self.history.setdefault(m, []).append(float(logs[m]))

        metric_types = ['val_f1', 'val_dir_acc', 'val_precision', 'val_recall']
        for m in metric_types:
            if m in logs:
                self.history.setdefault(m, []).append(float(logs[m]))

        # Update epoch-level plots
        self.update_loss_plot()
        self.update_metrics_plot()

        # Update progress bar
        self.progress_widget.value = self.epoch_count
        self.progress_widget.description = f"Epoch {self.epoch_count}/{self.total_epochs}"

    def update_loss_plot(self):
        """Update persistent loss plot with all loss types grouped"""
        epochs = list(range(1, self.epoch_count + 1))

        fig = go.Figure()

        # Training losses
        if 'loss' in self.history and self.history['loss']:
            fig.add_trace(go.Scatter(x=epochs, y=self.history['loss'], mode='lines', 
                                    name='Total Loss', line=dict(width=2, color='blue')))

        # Validation losses
        if 'val_loss' in self.history and self.history['val_loss']:
            fig.add_trace(go.Scatter(x=epochs, y=self.history['val_loss'], mode='lines', 
                                    name='Val Loss', line=dict(width=2, color='red', dash='dash')))

        # Point losses
        for loss_name in ['point_loss', 'val_point_loss']:
            if loss_name in self.history and self.history[loss_name]:
                fig.add_trace(go.Scatter(x=epochs, y=self.history[loss_name], mode='lines', 
                                        name=loss_name, line=dict(width=1.5)))

        # Trend losses
        for loss_name in ['trend_loss', 'val_trend_loss', 'local_trend_loss', 'val_local_trend_loss',
                         'global_trend_loss', 'val_global_trend_loss', 'extended_trend_loss', 'val_extended_trend_loss',
                         'dir_loss', 'val_dir_loss', 'reg_loss', 'val_reg_loss', 'vol_loss', 'val_vol_loss', 'var_nll', 'val_var_nll']:
            if loss_name in self.history and self.history[loss_name]:
                fig.add_trace(go.Scatter(x=epochs, y=self.history[loss_name], mode='lines', 
                                        name=loss_name, line=dict(width=1), opacity=0.7))

        fig.update_layout(
            title='Training Progress: All Losses',
            xaxis_title='Epoch',
            yaxis_title='Loss Value',
            hovermode='x unified',
            height=500,
            width=1200,
            template='plotly_dark',
            legend=dict(x=1.01, y=1, xanchor='left', yanchor='top'),
            showlegend=True,
        )

        with self.loss_output:
            clear_output(wait=True)
            display(fig)

    def update_metrics_plot(self):
        """Update persistent metrics plot with all metric types"""
        epochs = list(range(1, self.epoch_count + 1))

        fig = go.Figure()

        # Directional metrics
        for metric_name in ['val_f1', 'val_dir_acc', 'val_precision', 'val_recall']:
            if metric_name in self.history and self.history[metric_name]:
                fig.add_trace(go.Scatter(x=epochs, y=self.history[metric_name], mode='lines+markers', 
                                        name=metric_name, line=dict(width=2)))

        # Add 50% threshold baseline
        if self.epoch_count > 0:
            fig.add_hline(y=0.5, line_dash="dash", line_color="gray")

        fig.update_layout(
            title='Training Progress: Validation Metrics',
            xaxis_title='Epoch',
            yaxis_title='Metric Value',
            yaxis=dict(range=[0, 1]),
            hovermode='x unified',
            height=400,
            width=1200,
            template='plotly_dark',
            legend=dict(x=1.01, y=1, xanchor='left', yanchor='top'),
            showlegend=True,
        )

        with self.metrics_output:
            clear_output(wait=True)
            display(fig)

    def update_batch_plot(self):
        """Update a per-epoch batch-level plot inside batch_metrics_output."""
        batches = self.batch_history.get('batch', [])
        if not batches:
            return

        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08,
                            subplot_titles=("Batch Loss (per-batch)", "Batch Metrics (per-batch)"))

        # Loss lines
        loss_vals = self.batch_history.get('loss', [])
        if loss_vals:
            fig.add_trace(go.Scatter(x=batches, y=loss_vals, mode='lines+markers', name='batch_loss', line=dict(color='blue')), row=1, col=1)
        val_loss_vals = self.batch_history.get('val_loss', [])
        if val_loss_vals:
            fig.add_trace(go.Scatter(x=batches, y=val_loss_vals, mode='lines+markers', name='batch_val_loss', line=dict(color='red', dash='dash')), row=1, col=1)

        # Metric lines: dir_acc and f1 (plus optional validation counterparts if present)
        dir_vals = self.batch_history.get('dir_acc', [])
        if dir_vals:
            fig.add_trace(go.Scatter(x=batches, y=dir_vals, mode='lines+markers', name='dir_acc', line=dict(color='royalblue')), row=2, col=1)
        val_dir_vals = self.batch_history.get('val_dir_acc', [])
        if val_dir_vals:
            fig.add_trace(go.Scatter(x=batches, y=val_dir_vals, mode='lines+markers', name='val_dir_acc', line=dict(color='deepskyblue', dash='dash')), row=2, col=1)

        f1_vals = self.batch_history.get('f1', [])
        if f1_vals:
            fig.add_trace(go.Scatter(x=batches, y=f1_vals, mode='lines+markers', name='f1', line=dict(color='orange')), row=2, col=1)
        val_f1_vals = self.batch_history.get('val_f1', [])
        if val_f1_vals:
            fig.add_trace(go.Scatter(x=batches, y=val_f1_vals, mode='lines+markers', name='val_f1', line=dict(color='darkorange', dash='dash')), row=2, col=1)

        # Add baseline 50% line on metric subplot
        fig.update_yaxes(range=[0, 1], row=2, col=1)
        fig.add_hline(y=0.5, line_dash="dash", line_color="gray", row=2, col=1)

        fig.update_layout(
            height=500,
            width=1200,
            template='plotly_dark',
            hovermode='x unified',
            showlegend=True,
        )

        with self.batch_metrics_output:
            clear_output(wait=True)
            display(fig)


def limit_dataset_size(df, close_values, max_samples, lookback):
    required_rows = max_samples + lookback
    if len(df) > required_rows:
        df_limited = df.tail(required_rows).copy()
        close_limited = close_values[-required_rows:]
        print(f"Limited dataset to {len(df_limited):,} rows (most recent)")
        return df_limited, close_limited
    return df, close_values

def estimate_training_time(config, close_values, train_samples):
    batches_per_epoch = train_samples // config.BATCH_SIZE
    try:
        predictor = PricePredictor(config)
        base_model = predictor.build_model()
        test_model = CustomTrainModel(base_model=base_model, pred_scale=1.0, pred_mean=0.0, config=config,
                                      inputs=base_model.inputs, outputs=base_model.outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=config.LR)

        # Create sample batch
        close_np = np.array(close_values)
        start_indices = np.arange(0, min(config.BATCH_SIZE * config.WINDOW_STEP, len(close_np) - config.LOOKBACK - 1), config.WINDOW_STEP)
        X_batch = np.array([close_np[start:start + config.LOOKBACK] for start in start_indices[:config.BATCH_SIZE]])
        y_batch = np.array([close_np[start + config.LOOKBACK] for start in start_indices[:config.BATCH_SIZE]])[:, np.newaxis]
        last_close_batch = np.array([close_np[start + config.LOOKBACK - 1] for start in start_indices[:config.BATCH_SIZE]])[:, np.newaxis]
        extended_batch = np.array([[close_np[start + config.LOOKBACK + p] for p in config.EXTENDED_TREND_PERIODS] for start in start_indices[:config.BATCH_SIZE]])

        test_X = tf.convert_to_tensor(X_batch)
        test_y = tf.convert_to_tensor(y_batch)
        test_last_close = tf.convert_to_tensor(last_close_batch)
        test_extended = tf.convert_to_tensor(extended_batch)

        start_time = time.time()
        with tqdm(total=3, desc="Benchmarking", leave=False) as pbar:
            for _ in range(3):
                with tf.GradientTape() as tape:
                    predictions = test_model(test_X, training=True)
                    loss_components = test_model.custom_loss(test_X, test_y, predictions, test_last_close, test_extended)
                    total_loss = loss_components[0]
                grads = tape.gradient(total_loss, test_model.trainable_variables)
                optimizer.apply_gradients(zip(grads, test_model.trainable_variables))
                pbar.update(1)
        elapsed = time.time() - start_time
        global estimated_time_per_batch
        estimated_time_per_batch = (elapsed / 3) * (config.BATCH_SIZE / len(X_batch))
    except Exception as e:
        print(f"Benchmarking failed: {e}")
        estimated_time_per_batch = 0.15 if tf.config.list_physical_devices('GPU') else 2.0

    time_per_epoch = batches_per_epoch * estimated_time_per_batch
    global total_time
    total_time = time_per_epoch * config.EPOCHS
    return estimated_time_per_batch, total_time

def run_evaluation(model, scaler, X_test_seq, y_test, y_pred, last_close_test):
    """Run evaluation and display results"""
    # Interactive evaluation plots
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Actual vs Predicted (Last 200)', 'Error Distribution'))
    fig.add_trace(go.Scatter(y=y_test[-200:], mode='lines', name='Actual'), row=1, col=1)
    fig.add_trace(go.Scatter(y=y_pred[-200:], mode='lines', name='Predicted'), row=1, col=1)
    errors = y_test - y_pred
    fig.add_trace(go.Histogram(x=errors, nbinsx=50, name='Errors'), row=1, col=2)
    fig.update_layout(height=400, width=1200, template='plotly_dark')
    display(fig)

    # Metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    pred_dir = (y_pred - last_close_test) > 0
    true_dir = (y_test - last_close_test) > 0
    dir_acc = np.mean(pred_dir == true_dir)

    print("\nEvaluation Metrics:")
    print(f"   MSE: {mse:.4f}")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   R-squared: {r2:.4f}")
    print(f"   MAPE: {mape:.4f}")
    print(f"   Direction Accuracy: {dir_acc:.2%}")

# ============================================================================
# Main Pipeline
# ============================================================================
print("Training and Inference Cell")
print("=" * 80)

config = Config()

# Check weights
weights_exist = os.path.exists(config.MODEL_PATH)
force = False  # Set to True to force retraining

if not weights_exist or force:
    print("Running Training Pipeline")
    print("-" * 80)
    
    # Setup GPU
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"GPU: {len(gpus)} detected")
        for gpu in gpus:
            print(f"   {gpu.name}")
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        print("Running on CPU")

    # Load and prepare data
    data_processor = DataProcessor(config)
    df_full, close_full = data_processor.load_and_prepare_data()
    df, close_values = limit_dataset_size(df_full, close_full, config.MAX_SEQUENCE_COUNT, config.LOOKBACK)

    # Calculate dataset statistics
    max_sequences = len(close_values) - config.LOOKBACK - max(config.EXTENDED_TREND_PERIODS)
    expected_samples = max_sequences // config.WINDOW_STEP
    train_samples = int(expected_samples * 0.8)
    test_samples = expected_samples - train_samples

    print("\nDataset Statistics:")
    print(f"   Total rows: {len(df):,}")
    print(f"   Expected samples: {expected_samples:,}")
    print(f"   Training samples: {train_samples:,}")
    print(f"   Test samples: {test_samples:,}")

    # Benchmark and estimate training time
    print("\nEstimating training time...")
    estimated_time_per_batch, total_time = estimate_training_time(config, close_values, train_samples)
    
    print("\nTraining Time Estimation:")
    print(f"   Time per batch: {estimated_time_per_batch:.3f}s")
    print(f"   Est. time per epoch: {(train_samples // config.BATCH_SIZE * estimated_time_per_batch) / 60:.1f} min")
    print(f"   Est. total time: {total_time / 3600:.2f}h")

    print("\n" + "=" * 80)
    print("Training Controls & Progress")
    print("-" * 80)

    # Create control buttons
    pause_btn = Button(description="Pause", button_style='warning')
    resume_btn = Button(description="Resume", button_style='info')
    stop_btn = Button(description="Stop", button_style='danger')

    def on_pause(b):
        global pause_training
        pause_training = True
        pause_btn.disabled = True
        resume_btn.disabled = False

    def on_resume(b):
        global pause_training
        pause_training = False
        pause_btn.disabled = False
        resume_btn.disabled = True

    def on_stop(b):
        global stop_training
        stop_training = True
        stop_btn.disabled = True

    pause_btn.on_click(on_pause)
    resume_btn.on_click(on_resume)
    stop_btn.on_click(on_stop)
    resume_btn.disabled = True  # Start with resume disabled

    # Create persistent progress bar
    progress_bar = FloatProgress(value=0, min=0, max=config.EPOCHS, description='Epoch 0/100')
    progress_bar.style.bar_color = '#00aa00'

    # Create Output widgets for persistent plot display
    loss_output = Output()
    metrics_output = Output()
    # Batch-level plot output only (no text output)
    batch_metrics_output = Output()

    # Display control panel and progress
    controls_panel = HBox([pause_btn, resume_btn, stop_btn])
    display(controls_panel)
    display(progress_bar)

    # Train
    live_cb = InteractivePlotCallback(
        config, loss_output, metrics_output, progress_bar, config.EPOCHS,
        batch_metrics_output=batch_metrics_output,
    )
    original_load = DataProcessor.load_and_prepare_data
    DataProcessor.load_and_prepare_data = lambda self: (df, close_values)

    # Display plot containers
    print("\nReal-time Training Plots (updating each epoch)...")
    print("-" * 80)
    display(loss_output)
    display(metrics_output)
    # Display per-batch plot
    display(batch_metrics_output)

    try:
        model, scaler, X_test_seq, y_test, y_pred, last_close_test, history, ext = train_model(
            extra_callbacks=[live_cb], epochs=None, force=0, calibrate=0
        )
        if model:
            print("\nTraining completed successfully!")
            print("\n" + "=" * 80)
            print("Final Evaluation")
            print("-" * 80)
            run_evaluation(model, scaler, X_test_seq, y_test, y_pred, last_close_test)
    except Exception as e:
        print(f"\nTraining Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        DataProcessor.load_and_prepare_data = original_load
        pause_btn.disabled = True
        resume_btn.disabled = True
        stop_btn.disabled = True

else:
    print("Running Evaluation Pipeline")
    print("-" * 80)
    try:
        # Rebuild the model architecture first
        scaler = joblib.load(config.SCALER_PATH)
        data_processor = DataProcessor(config)
        df_full, close_full = data_processor.load_and_prepare_data()
        df, close_values = limit_dataset_size(df_full, close_full, config.MAX_SEQUENCE_COUNT, config.LOOKBACK)
        
        (X_train_seq, y_train_scaled, last_close_train, extended_trends_train,
         X_test_seq, y_test_scaled, last_close_test, extended_trends_test,
         y_train, y_test, target_scaler) = data_processor.prepare_datasets(df, close_values)
        
        print(f"X_test_seq shape: {X_test_seq.shape}")
        print(f"y_test shape: {y_test.shape}")
        
        # Rebuild model architecture
        predictor = PricePredictor(config)
        base_model = predictor.build_model()
        pred_scale = np.std(y_train) if np.std(y_train) > 0 else 1.0
        pred_mean = np.mean(y_train)
        model = CustomTrainModel(
            base_model=base_model,
            pred_scale=pred_scale,
            pred_mean=pred_mean,
            lambda_point=config.LAMBDA_POINT,
            lambda_local_trend=config.LAMBDA_LOCAL_TREND,
            lambda_global_trend=config.LAMBDA_GLOBAL_TREND,
            lambda_extended_trend=config.LAMBDA_EXTENDED_TREND,
            lambda_dir=config.LAMBDA_DIR,
            config=config,
            inputs=base_model.inputs,
            outputs=base_model.outputs
        )
        
        # Load the saved weights
        model.load_weights(config.MODEL_PATH)
        print("Model loaded successfully")
        
        # Run Multi-Horizon Evaluation on test set
        print("\nRunning Multi-Horizon Evaluation on test set...")
        batch_size = config.BATCH_SIZE
        y_pred_all_horizons = []  # Will store all 3 horizons
        
        print(f"X_test_seq shape: {X_test_seq.shape}")
        print(f"y_test shape: {y_test.shape}")
        print(f"Total test samples: {len(y_test)}")
        
        # Collect predictions from all batches for all horizons
        for i in range(0, len(X_test_seq), batch_size):
            batch_end = min(i + batch_size, len(X_test_seq))
            X_batch = X_test_seq[i:batch_end]
            
            # Convert to tensor
            X_batch_tf = tf.convert_to_tensor(X_batch, dtype=tf.float32)
            
            # Run prediction - returns (price[B,3], direction[B,1], variance[B,1])
            pred_batch = model(X_batch_tf, training=False)
            y_pred_batch, _, _ = pred_batch  # y_pred_batch shape: (batch_size, 3)
            
            y_pred_all_horizons.append(y_pred_batch.numpy())
        
        # Concatenate all predictions - shape: (total_samples, 3)
        y_pred_all = np.concatenate(y_pred_all_horizons, axis=0)
        
        # Trim to exact test size
        y_pred_all = y_pred_all[:len(y_test)]
        
        print(f"Total predictions shape: {y_pred_all.shape}")
        print(f"Expected: ({len(y_test)}, 3)")
        assert y_pred_all.shape[0] == len(y_test), f"Shape mismatch: {y_pred_all.shape[0]} != {len(y_test)}"
        
        # Define horizon names and weights
        horizon_names = ["1-min (Primary)", "5-min", "15-min"]
        horizon_weights = [config.LAMBDA_SHORT, config.LAMBDA_POINT, config.LAMBDA_LONG]
        
        print("\n" + "="*80)
        print("MULTI-HORIZON EVALUATION RESULTS")
        print("="*80)
        
        # Evaluate each horizon separately
        horizon_metrics = {}
        for h_idx in range(y_pred_all.shape[1]):
            print(f"\nHorizon {h_idx}: {horizon_names[h_idx]} (weight: {horizon_weights[h_idx]:.2f})")
            print("-" * 60)
            
            # Extract predictions for this horizon and reshape for inverse transform
            y_pred_h_scaled = y_pred_all[:, h_idx:h_idx+1]  # Shape: (N, 1)
            y_pred_h = target_scaler.inverse_transform(y_pred_h_scaled).ravel()
            
            # Calculate regression metrics
            mse_h = mean_squared_error(y_test, y_pred_h)
            rmse_h = np.sqrt(mse_h)
            r2_h = r2_score(y_test, y_pred_h)
            mape_h = mean_absolute_percentage_error(y_test, y_pred_h)
            
            # Calculate direction metrics
            pred_dir_h = (y_pred_h - last_close_test) > 0
            true_dir = (y_test - last_close_test) > 0
            dir_acc_h = accuracy_score(true_dir, pred_dir_h)
            f1_h = f1_score(true_dir, pred_dir_h, zero_division=0)
            
            # Store for later use
            horizon_metrics[h_idx] = {
                'y_pred': y_pred_h,
                'mse': mse_h,
                'rmse': rmse_h,
                'r2': r2_h,
                'mape': mape_h,
                'dir_acc': dir_acc_h,
                'f1': f1_h
            }
            
            print(f"  Regression Metrics:")
            print(f"    MSE:   {mse_h:.6f}")
            print(f"    RMSE:  {rmse_h:.6f}")
            print(f"    R2:    {r2_h:.6f}")
            print(f"    MAPE:  {mape_h:.4f}%")
            print(f"\n  Direction Metrics:")
            print(f"    Accuracy: {dir_acc_h:.4f} ({dir_acc_h*100:.2f}%)")
            print(f"    F1-Score: {f1_h:.4f}")
        
        # Use primary horizon (1-min) for main comparison
        y_pred = horizon_metrics[0]['y_pred']
        
        print("\n" + "="*80)
        print("SUMMARY: Primary Horizon (1-min) vs Others")
        print("="*80)
        
        # Create comparison table
        print(f"\n{'Metric':<15} {'1-min':<15} {'5-min':<15} {'15-min':<15}")
        print("-" * 60)
        
        for metric_name in ['mse', 'rmse', 'r2', 'mape', 'dir_acc', 'f1']:
            values = [horizon_metrics[h][metric_name] for h in range(3)]
            if metric_name in ['mse', 'rmse', 'mape']:
                print(f"{metric_name:<15} {values[0]:<15.6f} {values[1]:<15.6f} {values[2]:<15.6f}")
            else:
                print(f"{metric_name:<15} {values[0]:<15.4f} {values[1]:<15.4f} {values[2]:<15.4f}")
        
        # Multi-horizon visualization (expanded to include 15-min plot)
        print("\n" + "="*80)
        print("Creating Multi-Horizon Comparison Plots...")
        print("="*80)
        
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=(
                'Actual vs 1-min', 
                'Actual vs 5-min',
                'Actual vs 15-min',
                'Error Distribution (1-min)', 
                'Direction Accuracy by Horizon',
                ''
            ),
            specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
                   [{"type": "histogram"}, {"type": "bar"}, {"type": "scatter"}]]
        )
        
        # Plot 1: Actual vs 1-min predictions (last 200 samples)
        fig.add_trace(
            go.Scatter(y=y_test[-200:], mode='lines', name='Actual', line=dict(color='blue')),
            row=1, col=1,
        )
        fig.add_trace(
            go.Scatter(y=horizon_metrics[0]['y_pred'][-200:], mode='lines', name='1-min Pred', 
                      line=dict(color='red', dash='dash')),
            row=1, col=1,
        )
        
        # Plot 2: Actual vs 5-min predictions (last 200 samples)
        fig.add_trace(
            go.Scatter(y=y_test[-200:], mode='lines', name='Actual', line=dict(color='blue'), showlegend=False),
            row=1, col=2,
        )
        fig.add_trace(
            go.Scatter(y=horizon_metrics[1]['y_pred'][-200:], mode='lines', name='5-min Pred', 
                      line=dict(color='green', dash='dash')),
            row=1, col=2,
        )
        
        # Plot 3: Actual vs 15-min predictions (last 200 samples)
        fig.add_trace(
            go.Scatter(y=y_test[-200:], mode='lines', name='Actual', line=dict(color='blue'), showlegend=False),
            row=1, col=3,
        )
        fig.add_trace(
            go.Scatter(y=horizon_metrics[2]['y_pred'][-200:], mode='lines', name='15-min Pred', 
                      line=dict(color='orange', dash='dash')),
            row=1, col=3,
        )
        
        # Plot 4: Error distribution for primary horizon
        errors_h0 = y_test - horizon_metrics[0]['y_pred']
        fig.add_trace(
            go.Histogram(x=errors_h0, nbinsx=50, name='Errors', marker=dict(color='purple')),
            row=2, col=1,
        )
        
        # Plot 5: Direction accuracy comparison
        dir_accs = [horizon_metrics[h]['dir_acc'] for h in range(3)]
        fig.add_trace(
            go.Bar(x=horizon_names, y=dir_accs, name='Direction Accuracy',
                   marker=dict(color=['red', 'green', 'orange']), showlegend=False),
            row=2, col=2,
        )
        fig.add_hline(y=0.5, line_dash="dash", line_color="gray", row=2, col=2, annotation_text="50% Baseline")
        
        # (Optional empty subplot placeholder row=2,col=3)
        fig.add_trace(
            go.Scatter(x=[0], y=[0], mode='text', text=[''], showlegend=False),
            row=2, col=3,
        )
        
        # Axis titles
        fig.update_xaxes(title_text="Sample", row=1, col=1)
        fig.update_xaxes(title_text="Sample", row=1, col=2)
        fig.update_xaxes(title_text="Sample", row=1, col=3)
        fig.update_xaxes(title_text="Error Value", row=2, col=1)
        fig.update_xaxes(title_text="Horizon", row=2, col=2)
        fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
        fig.update_yaxes(title_text="Price (USD)", row=1, col=2)
        fig.update_yaxes(title_text="Price (USD)", row=1, col=3)
        fig.update_yaxes(title_text="Frequency", row=2, col=1)
        fig.update_yaxes(title_text="Accuracy", row=2, col=2)
        fig.update_yaxes(range=[0, 1], row=2, col=2)
        
        fig.update_layout(height=800, width=1600, template='plotly_dark', showlegend=True)
        display(fig)
        
        # Run evaluation plots for primary horizon
        run_evaluation(model, target_scaler, X_test_seq, y_test, y_pred, last_close_test)
        
    except Exception as e:
        print("Evaluation failed: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "=" * 80)
print("Training and Inference Cell Complete")
print("=" * 80)
    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

print("="*80)
print("DIRECTION HEAD ACCURACY EVALUATION")
print("="*80)

try:
    # Extract direction predictions from model
    print("\nExtracting direction predictions from test set...")
    
    batch_size = config.BATCH_SIZE
    direction_preds_all = []  # Raw direction logits/probabilities
    direction_binary_all = []  # Binary direction (0/1)
    
    for i in range(0, len(X_test_seq), batch_size):
        batch_end = min(i + batch_size, len(X_test_seq))
        X_batch = X_test_seq[i:batch_end]
        
        # Convert to tensor
        X_batch_tf = tf.convert_to_tensor(X_batch, dtype=tf.float32)
        
        # Run prediction - returns (price[B,3], direction[B,1], variance[B,1])
        pred_batch = model(X_batch_tf, training=False)
        _, direction_batch, _ = pred_batch  # direction_batch shape: (batch_size, 1)
        
        direction_preds_all.append(direction_batch.numpy())
        # Convert to binary (> 0.5)
        direction_binary = (direction_batch.numpy() > 0.5).astype(int).ravel()
        direction_binary_all.append(direction_binary)
    
    # Concatenate all predictions
    direction_probs = np.concatenate(direction_preds_all, axis=0).ravel()
    direction_preds_binary = np.concatenate(direction_binary_all, axis=0)
    
    # Trim to exact test size
    direction_probs = direction_probs[:len(y_test)]
    direction_preds_binary = direction_preds_binary[:len(y_test)]
    
    # Calculate true direction (1 = up, 0 = down)
    true_direction = (y_test - last_close_test).ravel() > 0
    true_direction_binary = true_direction.astype(int)
    
    print(f"Direction probabilities shape: {direction_probs.shape}")
    print(f"True direction shape: {true_direction_binary.shape}")
    
    # Calculate accuracy metrics
    accuracy = accuracy_score(true_direction_binary, direction_preds_binary)
    precision = precision_score(true_direction_binary, direction_preds_binary, zero_division=0)
    recall = recall_score(true_direction_binary, direction_preds_binary, zero_division=0)
    f1 = f1_score(true_direction_binary, direction_preds_binary, zero_division=0)
    
    # ROC-AUC score
    try:
        roc_auc = roc_auc_score(true_direction_binary, direction_probs)
    except:
        roc_auc = 0.0
    
    # Confusion matrix
    cm = confusion_matrix(true_direction_binary, direction_preds_binary)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print("\n" + "="*80)
    print("DIRECTION HEAD ACCURACY METRICS")
    print("="*80)
    
    print(f"\nPrimary Metrics:")
    print(f"  Accuracy:     {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision:    {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall:       {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score:     {f1:.4f}")
    print(f"  ROC-AUC:      {roc_auc:.4f}")
    
    print(f"\nSecondary Metrics:")
    print(f"  Sensitivity:  {sensitivity:.4f} ({sensitivity*100:.2f}%)")
    print(f"  Specificity:  {specificity:.4f} ({specificity*100:.2f}%)")
    
    print(f"\nConfusion Matrix:")
    print(f"  True Negatives (correct downs):  {tn}")
    print(f"  False Positives (wrong ups):     {fp}")
    print(f"  False Negatives (wrong downs):   {fn}")
    print(f"  True Positives (correct ups):    {tp}")
    
    print(f"\nClass Distribution (Test Set):")
    print(f"  Up   (1): {np.sum(true_direction_binary)} samples ({np.mean(true_direction_binary)*100:.2f}%)")
    print(f"  Down (0): {len(true_direction_binary) - np.sum(true_direction_binary)} samples ({(1-np.mean(true_direction_binary))*100:.2f}%)")
    
    print(f"\nPredicted Class Distribution:")
    print(f"  Up   (1): {np.sum(direction_preds_binary)} samples ({np.mean(direction_preds_binary)*100:.2f}%)")
    print(f"  Down (0): {len(direction_preds_binary) - np.sum(direction_preds_binary)} samples ({(1-np.mean(direction_preds_binary))*100:.2f}%)")
    
    # Direction probability statistics
    print(f"\nDirection Probability Statistics:")
    print(f"  Mean:  {np.mean(direction_probs):.4f}")
    print(f"  Std:   {np.std(direction_probs):.4f}")
    print(f"  Min:   {np.min(direction_probs):.4f}")
    print(f"  Max:   {np.max(direction_probs):.4f}")
    print(f"  Median: {np.median(direction_probs):.4f}")
    
    # Create comprehensive visualization
    print("\n" + "="*80)
    print("Creating Direction Head Accuracy Visualizations...")
    print("="*80)
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'ROC Curve',
            'Confusion Matrix Heatmap',
            'Direction Probability Distribution',
            'Accuracy Metrics Comparison'
        ),
        specs=[[{"type": "scatter"}, {"type": "heatmap"}],
               [{"type": "histogram"}, {"type": "bar"}]]
    )
    
    # Plot 1: ROC Curve
    fpr, tpr, _ = roc_curve(true_direction_binary, direction_probs)
    fig.add_trace(
        go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC={roc_auc:.3f})',
                   line=dict(color='blue', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random (AUC=0.5)',
                   line=dict(color='gray', dash='dash')),
        row=1, col=1
    )
    
    # Plot 2: Confusion Matrix - Fixed heatmap
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_text = [[f'{cm[i, j]}<br>({cm_normalized[i, j]:.1%})' for j in range(2)] for i in range(2)]
    
    fig.add_trace(
        go.Heatmap(z=cm_normalized, x=['Predicted Down', 'Predicted Up'], 
                   y=['Actual Down', 'Actual Up'],
                   text=cm_text, texttemplate='%{text}',
                   colorscale='Blues', showscale=True,
                   hovertemplate='%{y}, %{x}<br>Count: %{customdata}<extra></extra>',
                   customdata=cm),
        row=1, col=2
    )
    
    # Plot 3: Direction Probability Distribution
    # Separate by true class
    up_probs = direction_probs[true_direction_binary == 1]
    down_probs = direction_probs[true_direction_binary == 0]
    
    fig.add_trace(
        go.Histogram(x=up_probs, name='True Up (Target=1)', nbinsx=30, opacity=0.7,
                     marker=dict(color='green')),
        row=2, col=1
    )
    fig.add_trace(
        go.Histogram(x=down_probs, name='True Down (Target=0)', nbinsx=30, opacity=0.7,
                     marker=dict(color='red')),
        row=2, col=1
    )
    fig.add_vline(x=0.5, line_dash="dash", line_color="black", row=2, col=1,
                  annotation_text="Decision Boundary")
    
    # Plot 4: Metrics Comparison
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'Specificity', 'Sensitivity', 'F1-Score']
    metrics_values = [accuracy, precision, recall, specificity, sensitivity, f1]
    colors = ['blue' if v >= 0.5 else 'red' for v in metrics_values]
    
    fig.add_trace(
        go.Bar(x=metrics_names, y=metrics_values, name='Metrics',
               marker=dict(color=colors), text=[f'{v:.3f}' for v in metrics_values],
               textposition='outside'),
        row=2, col=2
    )
    fig.add_hline(y=0.5, line_dash="dash", line_color="gray", row=2, col=2,
                  annotation_text="50% Baseline")
    
    # Update axes
    fig.update_xaxes(title_text="False Positive Rate", row=1, col=1)
    fig.update_yaxes(title_text="True Positive Rate", row=1, col=1)
    fig.update_xaxes(title_text="Predicted Class", row=1, col=2)
    fig.update_yaxes(title_text="Actual Class", row=1, col=2)
    fig.update_xaxes(title_text="Direction Probability", row=2, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_yaxes(title_text="Score", range=[0, 1], row=2, col=2)
    
    fig.update_layout(height=900, width=1400, template='plotly_dark', showlegend=True,
                     title_text="Direction Head Accuracy Analysis")
    display(fig)
    
    # Additional detailed analysis plot
    print("\nCreating prediction confidence analysis...")
    
    # Correct vs incorrect predictions
    correct_mask = (direction_preds_binary == true_direction_binary)
    correct_probs = direction_probs[correct_mask]
    incorrect_probs = direction_probs[~correct_mask]
    
    fig2 = go.Figure()
    
    fig2.add_trace(go.Histogram(x=correct_probs, name=f'Correct Predictions (n={len(correct_probs)})',
                                nbinsx=30, opacity=0.7, marker=dict(color='green')))
    fig2.add_trace(go.Histogram(x=incorrect_probs, name=f'Incorrect Predictions (n={len(incorrect_probs)})',
                                nbinsx=30, opacity=0.7, marker=dict(color='red')))
    
    fig2.update_layout(
        title='Direction Prediction Confidence Distribution',
        xaxis_title='Predicted Probability',
        yaxis_title='Frequency',
        template='plotly_dark',
        height=500, width=1200,
        barmode='overlay'
    )
    display(fig2)
    
    # Summary statistics table
    print("\n" + "="*80)
    print("DIRECTION HEAD PERFORMANCE SUMMARY")
    print("="*80)
    
    summary_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'Specificity', 'Sensitivity'],
        'Value': [accuracy, precision, recall, f1, roc_auc, specificity, sensitivity],
        'Percentage': [f'{v*100:.2f}%' for v in [accuracy, precision, recall, f1, roc_auc, specificity, sensitivity]]
    })
    
    print("\n" + summary_df.to_string(index=False))
    
    # Interpretation
    print("\n" + "="*80)
    print("INTERPRETATION")
    print("="*80)
    
    if accuracy >= 0.60:
        print(f"\n✅ GOOD: Direction head accuracy ({accuracy*100:.2f}%) exceeds 60% baseline")
    elif accuracy >= 0.55:
        print(f"\n⚠️  ACCEPTABLE: Direction head accuracy ({accuracy*100:.2f}%) near 55-60% target range")
    else:
        print(f"\n❌ BELOW TARGET: Direction head accuracy ({accuracy*100:.2f}%) below 55% minimum")
    
    if roc_auc >= 0.60:
        print(f"✅ ROC-AUC ({roc_auc:.4f}) shows good discrimination ability")
    else:
        print(f"⚠️  ROC-AUC ({roc_auc:.4f}) indicates limited discrimination")
    
    if precision >= recall:
        print(f"✅ High precision ({precision:.2%}) - few false positives (conservative predictions)")
    else:
        print(f"⚠️  Low precision ({precision:.2%}) - many false positives (liberal predictions)")
    
    avg_confidence = np.mean(np.max(np.vstack([1-direction_probs, direction_probs]), axis=0))
    print(f"\n📊 Average prediction confidence: {avg_confidence:.2%}")
    
    if len(incorrect_probs) > 0:
        avg_incorrect_confidence = np.mean(incorrect_probs)
        print(f"⚠️  Average confidence on incorrect predictions: {avg_incorrect_confidence:.2%}")
    
except Exception as e:
    print(f"\n❌ Direction evaluation failed: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("Direction Head Evaluation Complete")
print("="*80)

In [ ]:
# ============================================================================
# TRADE DATACLASS AND COMPLETE MULTI-HEAD STRATEGY PIPELINE
# ============================================================================
from dataclasses import dataclass
from typing import List

@dataclass
class Trade:
    """Professional trade representation"""
    entry_bar: int           # Bar where trade entered
    exit_bar: int            # Bar where trade exited
    entry_price: float       # Entry price at entry_bar
    exit_price: float        # Exit price at exit_bar
    trade_type: str          # 'LONG' or 'SHORT'
    bars_held: int           # Number of bars held (exit_bar - entry_bar)
    profit: float            # Absolute profit
    profit_pct: float        # Percentage profit
    exit_reason: str         # 'SPIKE', 'REV', 'TIME'
    
    # Calculated target levels (not executed, just reference)
    tp1_price: float = None
    tp2_price: float = None
    sl_price: float = None
    
    def __post_init__(self):
        """Calculate target levels after initialization"""
        if self.trade_type == 'LONG':
            self.tp1_price = self.entry_price + (self.entry_price * 0.005)   # +0.5%
            self.tp2_price = self.entry_price + (self.entry_price * 0.015)   # +1.5%
            self.sl_price = self.entry_price - (self.entry_price * 0.01)     # -1%
        else:  # SHORT
            self.tp1_price = self.entry_price - (self.entry_price * 0.005)   # -0.5%
            self.tp2_price = self.entry_price - (self.entry_price * 0.015)   # -1.5%
            self.sl_price = self.entry_price + (self.entry_price * 0.01)     # +1%
    
    def is_win(self) -> bool:
        return self.profit > 0
    
    def __repr__(self):
        return (f"Trade({self.trade_type} @{self.entry_bar}-{self.exit_bar}, "
                f"${self.entry_price:.2f}→${self.exit_price:.2f}, "
                f"{self.profit:+.2f}pts, {self.exit_reason})")

# ============================================================================
# MULTI-HEAD STRATEGY PIPELINE - COMPLETE EXECUTION
# ============================================================================
# Phases 1-6: Full end-to-end execution with all 3 model heads
# ============================================================================

print("="*80)
print("MULTI-HEAD STRATEGY PIPELINE - COMPLETE EXECUTION")
print("="*80)

# Phase 1: Extract all 3 heads from model
print("\n[PHASE 1] Data Extraction...")
batch_size = config.BATCH_SIZE
direction_preds_all, price_preds_all, variance_preds_all = [], [], []

for i in range(0, len(X_test_seq), batch_size):
    batch_end = min(i + batch_size, len(X_test_seq))
    X_batch_tf = tf.convert_to_tensor(X_test_seq[i:batch_end], dtype=tf.float32)
    price_batch, direction_batch, variance_batch = model(X_batch_tf, training=False)
    direction_preds_all.append(direction_batch.numpy())
    price_preds_all.append(price_batch.numpy())
    variance_preds_all.append(variance_batch.numpy())

direction_probs = np.concatenate(direction_preds_all, axis=0).ravel()[:len(y_test)]
price_preds = np.concatenate(price_preds_all, axis=0)[:len(y_test)]
variance_raw = np.concatenate(variance_preds_all, axis=0).ravel()[:len(y_test)]

price_1min = target_scaler.inverse_transform(price_preds[:, 0:1]).ravel()
price_5min = target_scaler.inverse_transform(price_preds[:, 1:2]).ravel()
price_15min = target_scaler.inverse_transform(price_preds[:, 2:3]).ravel()

print(f"✓ Extracted 3 heads: {direction_probs.shape}, {price_preds.shape}, {variance_raw.shape}")

# Phase 2: Helper functions
print("\n[PHASE 2] Helper Functions...")

def calculate_confidence(var, eps=1e-7): return 1.0 / (1.0 + np.asarray(var) + eps)
def calculate_signal_strength(d, c): return np.asarray(d) * np.asarray(c)
def normalize_variance(v, m, s, eps=1e-7):
    return np.where(s < eps, 0.0, (v - m) / (s + eps))
def check_multi_horizon_agreement(preds, curr, thresh=0.67):
    preds = np.asarray(preds)
    up = np.sum(preds > curr)
    return max(up, len(preds) - up) / len(preds) >= thresh, max(up, len(preds) - up) / len(preds)
def detect_variance_spike(v, m, thresh=2.0, eps=1e-7): return v > thresh * (m + eps)

print("✓ 8 helper functions defined")

# Phase 3: Calculate metrics
print("\n[PHASE 3] Calculate Metrics...")
window = 20
var_mean = np.convolve(variance_raw, np.ones(window)/window, mode='same')
var_std = pd.Series(variance_raw).rolling(window, center=True).std().values
confidence = calculate_confidence(variance_raw)
signal_str = calculate_signal_strength(direction_probs, confidence)

dir_1m = direction_probs
dir_5m = pd.Series(direction_probs).rolling(5, center=True).mean().fillna(method='bfill').fillna(method='ffill').values
dir_15m = pd.Series(direction_probs).rolling(15, center=True).mean().fillna(method='bfill').fillna(method='ffill').values
weighted_sig = 0.2 * dir_1m + 0.3 * dir_5m + 0.5 * dir_15m

agreement = np.array([check_multi_horizon_agreement([price_1min[i], price_5min[i], price_15min[i]], y_test[i])[1] for i in range(len(y_test))])
var_normalized = normalize_variance(variance_raw, var_mean, var_std + 1e-7)

print(f"✓ Metrics: confidence={np.mean(confidence):.4f}, weighted_sig={np.mean(weighted_sig):.4f}")

# Phase 4: Create data feed
print("\n[PHASE 4] Create Data Feed...")
backtest_data = pd.DataFrame({
    'open': y_test, 'high': y_test * 1.001, 'low': y_test * 0.999, 'close': y_test, 'volume': np.ones(len(y_test)) * 1000,
    'dir_1m': dir_1m, 'dir_5m': dir_5m, 'dir_15m': dir_15m,
    'price_1m': price_1min, 'price_5m': price_5min, 'price_15m': price_15min,
    'variance': variance_raw, 'confidence': confidence, 'signal_str': signal_str,
    'weighted_sig': weighted_sig, 'agreement': agreement, 'var_norm': var_normalized,
})
print(f"✓ Data feed: {backtest_data.shape}")

# Phase 5: Strategy backtest with Trade dataclass
print("\n[PHASE 5] Backtest Strategy...")
trades: List[Trade] = []
in_pos = False
ent_bar = None
ent_price = None
ent_sig = None
pos_type = None

for bar in range(len(backtest_data)):
    price = backtest_data['close'].iloc[bar]
    wsig = backtest_data['weighted_sig'].iloc[bar]
    conf = backtest_data['confidence'].iloc[bar]
    var = backtest_data['variance'].iloc[bar]
    
    if not in_pos:
        # Entry logic
        if wsig > 0.25 and conf > 0.35:
            in_pos = True
            ent_bar = bar
            ent_price = price
            ent_sig = wsig
            pos_type = 'LONG'
        elif wsig < 0.75 and conf > 0.35:
            in_pos = True
            ent_bar = bar
            ent_price = price
            ent_sig = wsig
            pos_type = 'SHORT'
    else:
        # Position management
        bars_held = bar - ent_bar
        can_exit = bars_held >= 3
        var_mean_curr = np.mean(backtest_data['variance'].iloc[max(0, bar-20):bar])
        var_spike = detect_variance_spike(var, var_mean_curr)
        sig_rev = (pos_type == 'LONG' and can_exit and wsig < 0.40) or (pos_type == 'SHORT' and can_exit and wsig > 0.60)
        time_exit = bars_held >= 100
        
        # Exit logic
        if var_spike or sig_rev or time_exit:
            exit_price = price
            
            # Calculate profit
            if pos_type == 'LONG':
                profit = exit_price - ent_price
            else:  # SHORT
                profit = ent_price - exit_price
            
            profit_pct = (profit / ent_price) * 100
            exit_reason = "SPIKE" if var_spike else ("REV" if sig_rev else "TIME")
            
            # Create Trade object
            trade = Trade(
                entry_bar=ent_bar,
                exit_bar=bar,
                entry_price=float(ent_price),
                exit_price=float(exit_price),
                trade_type=pos_type,
                bars_held=bars_held,
                profit=float(profit),
                profit_pct=float(profit_pct),
                exit_reason=exit_reason
            )
            
            trades.append(trade)
            in_pos = False

print(f"✓ Backtest complete: {len(trades)} trades")

# Phase 6: Analysis
print("\n[PHASE 6] Performance Analysis...")
if trades:
    # Validate trade logic
    print("\n🔍 TRADE LOGIC VALIDATION")
    print("-" * 60)
    
    all_valid = True
    for i, trade in enumerate(trades):
        # Verify entry < exit
        if trade.entry_bar >= trade.exit_bar:
            print(f"❌ Trade {i}: Entry bar {trade.entry_bar} >= Exit bar {trade.exit_bar}")
            all_valid = False
        
        # Verify entry price vs TP/SL
        if trade.trade_type == 'LONG':
            if trade.entry_price > trade.tp1_price:
                print(f"❌ Trade {i}: Entry ${trade.entry_price:.2f} > TP1 ${trade.tp1_price:.2f}")
                all_valid = False
            if trade.entry_price < trade.sl_price:
                print(f"❌ Trade {i}: Entry ${trade.entry_price:.2f} < SL ${trade.sl_price:.2f}")
                all_valid = False
        else:  # SHORT
            if trade.entry_price < trade.tp1_price:
                print(f"❌ Trade {i}: Entry ${trade.entry_price:.2f} < TP1 ${trade.tp1_price:.2f}")
                all_valid = False
            if trade.entry_price > trade.sl_price:
                print(f"❌ Trade {i}: Entry ${trade.entry_price:.2f} > SL ${trade.sl_price:.2f}")
                all_valid = False
    
    if all_valid:
        print("✅ All trades validated successfully!")
    else:
        print("⚠️  Some trades have logical inconsistencies")
    
    # Statistics
    wins = [t for t in trades if t.is_win()]
    losses = [t for t in trades if not t.is_win()]
    wr = len(wins) / len(trades) if trades else 0
    
    print("\n📊 PERFORMANCE SUMMARY")
    print("-" * 60)
    print(f"Total Trades:        {len(trades)}")
    print(f"Wins/Losses:         {len(wins)}/{len(losses)} ({wr*100:.1f}% win rate)")
    print(f"Total Profit:        {sum(t.profit for t in trades):+.2f} points")
    print(f"Avg Trade:           {sum(t.profit for t in trades)/len(trades):+.2f} points ({sum(t.profit_pct for t in trades)/len(trades):+.2f}%)")
    print(f"Best/Worst:          {max(t.profit for t in trades):+.2f} / {min(t.profit for t in trades):+.2f}")
    print(f"Avg Hold Time:       {sum(t.bars_held for t in trades)/len(trades):.1f} bars")
    
    exit_reasons = {}
    for t in trades:
        exit_reasons[t.exit_reason] = exit_reasons.get(t.exit_reason, 0) + 1
    
    print(f"\nExit Reasons:")
    for reason, count in sorted(exit_reasons.items()):
        print(f"  {reason:6s}: {count} ({count/len(trades)*100:.1f}%)")
    
    # Visualization
    profit_data = [t.profit for t in trades]
    pct_data = [t.profit_pct for t in trades]
    cumsum_data = np.cumsum(profit_data)
    
    fig = make_subplots(rows=2, cols=2, subplot_titles=('P&L Distribution', 'Cumulative P&L', 'Exit Reasons', 'Profit %'))
    fig.add_trace(go.Histogram(x=profit_data, name='P&L', nbinsx=15), row=1, col=1)
    fig.add_trace(go.Scatter(y=cumsum_data, mode='lines+markers', name='Cumulative'), row=1, col=2)
    
    exit_counts = pd.Series(exit_reasons)
    fig.add_trace(go.Bar(x=exit_counts.index, y=exit_counts.values, name='Exits'), row=2, col=1)
    fig.add_trace(go.Histogram(x=pct_data, name='%', nbinsx=15), row=2, col=2)
    
    fig.update_xaxes(title_text="P&L (points)", row=1, col=1)
    fig.update_xaxes(title_text="Trade #", row=1, col=2)
    fig.update_xaxes(title_text="Exit Type", row=2, col=1)
    fig.update_xaxes(title_text="Profit %", row=2, col=2)
    fig.update_layout(height=700, width=1300, template='plotly_dark', showlegend=False)
    display(fig)
    
    # Top trades
    print("\n✅ Top 5 Wins:")
    top_wins = sorted(wins, key=lambda t: t.profit, reverse=True)[:5]
    for t in top_wins:
        print(f"  Bar {t.entry_bar:4d}→{t.exit_bar:4d}: {t.trade_type:5s} ${t.entry_price:9.2f}→${t.exit_price:9.2f} = {t.profit:+8.2f} pts ({t.profit_pct:+6.2f}%) [{t.exit_reason}]")
else:
    print("❌ No trades executed")

print("\n" + "="*80)
print("✅ COMPLETE PIPELINE EXECUTED SUCCESSFULLY")
print("="*80)
print("\nAll 6 Phases Complete:")
print("  ✓ Data extraction from 3 model heads")
print("  ✓ 8 helper functions implemented")
print("  ✓ 10+ metrics calculated")
print("  ✓ 17-channel data feed created")
print("  ✓ Strategy backtested with Trade dataclass")
print("  ✓ Performance analyzed and validated")
print("\nReady for deployment!")


In [ ]:
# ============================================================================
# COMPREHENSIVE THREE-PANEL TRADING VISUALIZATION (FIXED)
# ============================================================================
# Uses Trade dataclass for proper trade representation
# Panel 1: Price with Entry/Exit + Synchronized TP/SL on hover
# Panel 2: Learned Indicators with vertical hover line
# Panel 3: Portfolio Evolution with vertical hover line
# ============================================================================

print("="*80)
print("CREATING COMPREHENSIVE THREE-PANEL TRADING VISUALIZATION (FIXED)")
print("="*80)

# Create portfolio history tracking
print("\nTracing portfolio values through trades...")
portfolio_values = []
current_portfolio = 10000  # Starting cash

for bar_num in range(len(backtest_data)):
    # Update portfolio for each trade that closes at this bar
    for trade in trades:
        if trade.exit_bar == bar_num:
            current_portfolio += trade.profit
    portfolio_values.append(current_portfolio)

portfolio_values = np.array(portfolio_values)
portfolio_pnl = portfolio_values - 10000
bar_idx_list = list(range(len(backtest_data)))

print(f"✓ Portfolio history: {len(portfolio_values)} bars")
print(f"  Starting: $10,000")
print(f"  Final: ${portfolio_values[-1]:,.2f}")
print(f"  P&L: ${portfolio_pnl[-1]:+,.2f}")

# Separate trades by type
buy_trades = [t for t in trades if t.trade_type == 'LONG']
sell_trades = [t for t in trades if t.trade_type == 'SHORT']

print(f"✓ Buy trades: {len(buy_trades)}")
print(f"✓ Sell trades: {len(sell_trades)}")
print(f"✓ Total trades: {len(trades)}")

# Create the three-panel figure with shared X-axis
print("\nBuilding three-panel visualization...")

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=(
        'Panel 1: Price with Entry/Exit Orders (synchronized hover)',
        'Panel 2: Learned Indicators (synchronized hover)',
        'Panel 3: Portfolio Value Evolution (synchronized hover)'
    ),
    specs=[
        [{"secondary_y": False}],
        [{"secondary_y": False}],
        [{"secondary_y": False}]
    ]
)

# ============================================================================
# PANEL 1: PRICE WITH ENTRY/EXIT ORDERS (PROPER POSITIONING)
# ============================================================================
print("\n[Panel 1] Adding price and trade entry/exit visualization...")

# Price line
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=backtest_data['close'].values,
        mode='lines',
        name='Price',
        line=dict(color='white', width=2),
        hovertemplate='<b>Price</b><br>Bar: %{x}<br>Price: $%{y:.2f}<extra></extra>',
        xaxis='x1', yaxis='y1'
    ),
    row=1, col=1
)

# For each trade, draw entry marker, exit marker, and TP/SL levels
for i, trade in enumerate(trades):
    # Entry marker (at actual entry bar)
    fig.add_trace(
        go.Scatter(
            x=[trade.entry_bar],
            y=[trade.entry_price],
            mode='markers',
            marker=dict(
                size=10,
                color='green' if trade.trade_type == 'LONG' else 'red',
                symbol='triangle-up' if trade.trade_type == 'LONG' else 'triangle-down',
                line=dict(color='lightgreen' if trade.trade_type == 'LONG' else 'lightcoral', width=2)
            ),
            name='Entry' if i == 0 else '',
            showlegend=(i == 0),
            hovertemplate=(
                f'<b>ENTRY ({trade.trade_type})</b><br>'
                f'Bar: {trade.entry_bar}<br>'
                f'Price: ${trade.entry_price:.2f}<br>'
                f'TP1: ${trade.tp1_price:.2f}<br>'
                f'TP2: ${trade.tp2_price:.2f}<br>'
                f'SL: ${trade.sl_price:.2f}<extra></extra>'
            ),
            xaxis='x1', yaxis='y1'
        ),
        row=1, col=1
    )
    
    # Exit marker (at actual exit bar)
    exit_color = 'lime' if trade.is_win() else 'orange'
    fig.add_trace(
        go.Scatter(
            x=[trade.exit_bar],
            y=[trade.exit_price],
            mode='markers',
            marker=dict(
                size=10,
                color=exit_color,
                symbol='circle',
                line=dict(color='white', width=2)
            ),
            name='Exit' if i == 0 else '',
            showlegend=(i == 0),
            hovertemplate=(
                f'<b>EXIT ({trade.exit_reason})</b><br>'
                f'Bar: {trade.exit_bar}<br>'
                f'Price: ${trade.exit_price:.2f}<br>'
                f'Profit: {trade.profit:+.2f} pts ({trade.profit_pct:+.2f}%)<br>'
                f'Hold: {trade.bars_held} bars<extra></extra>'
            ),
            xaxis='x1', yaxis='y1'
        ),
        row=1, col=1
    )
    
    # Draw connection line between entry and exit (faint)
    fig.add_trace(
        go.Scatter(
            x=[trade.entry_bar, trade.exit_bar],
            y=[trade.entry_price, trade.exit_price],
            mode='lines',
            line=dict(color='gray', width=1, dash='dot'),
            showlegend=False,
            hoverinfo='skip',
            xaxis='x1', yaxis='y1'
        ),
        row=1, col=1
    )

# ============================================================================
# PANEL 2: LEARNED INDICATORS
# ============================================================================
print("\n[Panel 2] Adding learned indicators...")

# Direction head (1-min, 5-min, 15-min moving averages)
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=dir_1m,
        mode='lines',
        name='Direction 1-min',
        line=dict(color='cyan', width=1.5),
        opacity=0.7,
        hovertemplate='<b>Dir 1-min</b><br>Bar: %{x}<br>Value: %{y:.4f}<extra></extra>',
        xaxis='x2', yaxis='y2'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=dir_5m,
        mode='lines',
        name='Direction 5-min',
        line=dict(color='blue', width=1.5),
        opacity=0.7,
        hovertemplate='<b>Dir 5-min</b><br>Bar: %{x}<br>Value: %{y:.4f}<extra></extra>',
        xaxis='x2', yaxis='y2'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=dir_15m,
        mode='lines',
        name='Direction 15-min',
        line=dict(color='navy', width=1.5),
        opacity=0.7,
        hovertemplate='<b>Dir 15-min</b><br>Bar: %{x}<br>Value: %{y:.4f}<extra></extra>',
        xaxis='x2', yaxis='y2'
    ),
    row=2, col=1
)

# Confidence
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=confidence,
        mode='lines',
        name='Confidence',
        line=dict(color='yellow', width=2),
        hovertemplate='<b>Confidence</b><br>Bar: %{x}<br>Value: %{y:.4f}<extra></extra>',
        xaxis='x2', yaxis='y2'
    ),
    row=2, col=1
)

# Weighted signal
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=weighted_sig,
        mode='lines',
        name='Weighted Signal',
        line=dict(color='lime', width=2.5),
        hovertemplate='<b>Weighted Signal</b><br>Bar: %{x}<br>Value: %{y:.4f}<extra></extra>',
        xaxis='x2', yaxis='y2'
    ),
    row=2, col=1
)

# Signal strength (filled area)
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=signal_str,
        mode='lines',
        name='Signal Strength',
        line=dict(color='orange', width=2),
        fill='tozeroy',
        fillcolor='rgba(255, 165, 0, 0.2)',
        hovertemplate='<b>Signal Strength</b><br>Bar: %{x}<br>Value: %{y:.4f}<extra></extra>',
        xaxis='x2', yaxis='y2'
    ),
    row=2, col=1
)

# Add subtle reference lines for entry/exit thresholds (low opacity)
fig.add_hline(y=0.25, line_dash="dash", line_color="rgba(128,128,128,0.3)", row=2, col=1)
fig.add_hline(y=0.75, line_dash="dash", line_color="rgba(128,128,128,0.3)", row=2, col=1)

# ============================================================================
# PANEL 3: PORTFOLIO VALUE EVOLUTION
# ============================================================================
print("\n[Panel 3] Adding portfolio evolution...")

# Main portfolio line
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=portfolio_values,
        mode='lines',
        name='Portfolio Value',
        line=dict(color='white', width=3),
        fill='tozeroy',
        fillcolor='rgba(100, 200, 100, 0.1)',
        hovertemplate='<b>Portfolio Value</b><br>Bar: %{x}<br>Value: $%{y:.2f}<extra></extra>',
        xaxis='x3', yaxis='y3'
    ),
    row=3, col=1
)

# P&L line
fig.add_trace(
    go.Scatter(
        x=bar_idx_list,
        y=portfolio_pnl,
        mode='lines',
        name='P&L',
        line=dict(color='gold', width=2),
        hovertemplate='<b>P&L</b><br>Bar: %{x}<br>Value: $%{y:+.2f}<extra></extra>',
        xaxis='x3', yaxis='y3'
    ),
    row=3, col=1
)

# Breakeven line (subtle)
fig.add_hline(y=0, line_dash="dash", line_color="rgba(128,128,128,0.3)", row=3, col=1)

# Mark trade exit points on portfolio panel
for trade in trades:
    exit_pnl = portfolio_pnl[trade.exit_bar] if trade.exit_bar < len(portfolio_pnl) else portfolio_pnl[-1]
    color = 'lime' if trade.is_win() else 'orange'
    
    fig.add_trace(
        go.Scatter(
            x=[trade.exit_bar],
            y=[exit_pnl],
            mode='markers',
            marker=dict(
                size=8,
                color=color,
                symbol='diamond' if trade.is_win() else 'x'
            ),
            showlegend=False,
            hovertemplate=(
                f'<b>Trade Close ({trade.exit_reason})</b><br>'
                f'Bar: {trade.exit_bar}<br>'
                f'P&L: {trade.profit:+.2f} pts<br>'
                f'Cumulative: ${exit_pnl:+.2f}<extra></extra>'
            ),
            xaxis='x3', yaxis='y3'
        ),
        row=3, col=1
    )

# ============================================================================
# FORMATTING AND LAYOUT
# ============================================================================
print("\nFormatting layout...")

# Update axes labels
fig.update_xaxes(title_text="Bar Index (Time)", row=3, col=1)
fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
fig.update_yaxes(title_text="Indicator Value", row=2, col=1)
fig.update_yaxes(title_text="Portfolio Value (USD)", row=3, col=1)

# Set Y-axis ranges for better visibility
fig.update_yaxes(
    range=[backtest_data['close'].min() * 0.99, backtest_data['close'].max() * 1.01],
    row=1, col=1
)

fig.update_yaxes(
    range=[0, 1],
    row=2, col=1
)

fig.update_yaxes(
    range=[min(portfolio_values) - 500, max(portfolio_values) + 500],
    row=3, col=1
)

# Update layout with synchronized hover
fig.update_layout(
    title_text="<b>Comprehensive Trading Analysis Dashboard (Fixed)</b><br><sub>Synchronized hover across all panels | Entry: Triangles | Exit: Circles | Hover for details</sub>",
    height=1200,
    width=1600,
    template='plotly_dark',
    hovermode='x unified',  # KEY: synchronized hover across all subplots
    legend=dict(
        x=1.01,
        y=1,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(0,0,0,0.7)',
        bordercolor='white',
        borderwidth=1,
        font=dict(size=10)
    ),
    font=dict(size=11),
    margin=dict(l=80, r=150, t=120, b=80),
    showlegend=True
)

print("✓ Displaying interactive three-panel visualization...")
display(fig)

# Print summary
print("\n" + "="*80)
print("VISUALIZATION SUMMARY (FIXED)")
print("="*80)
print(f"\n📊 Panel 1 (Top): Price Chart with Trade Entry/Exit")
print(f"  • Price range: ${backtest_data['close'].min():.2f} - ${backtest_data['close'].max():.2f}")
print(f"  • Entry markers: Green triangles (LONG) / Red triangles (SHORT)")
print(f"  • Exit markers: Lime circles (WIN) / Orange X (LOSS)")
print(f"  • Connection: Faint dotted line from entry to exit")
print(f"  • Hover shows: Entry price, TP1/TP2, SL levels")

print(f"\n📈 Panel 2 (Middle): Learned Indicators")
print(f"  • Direction heads: 1-min (cyan), 5-min (blue), 15-min (navy)")
print(f"  • Confidence: Yellow line [0, 1]")
print(f"  • Weighted Signal: Lime line - primary entry signal")
print(f"  • Signal Strength: Orange filled area")
print(f"  • Entry thresholds: 0.25 (buy) and 0.75 (sell) - subtle gray dashed lines")

print(f"\n💰 Panel 3 (Bottom): Portfolio Evolution")
print(f"  • Portfolio Value: White line with green fill")
print(f"  • P&L: Gold line relative to starting $10,000")
print(f"  • Exit markers: Lime diamonds (wins), Orange X (losses)")
print(f"  • Final Value: ${portfolio_values[-1]:,.2f}")
print(f"  • Total P&L: ${portfolio_pnl[-1]:+,.2f}")

print(f"\n🔗 Synchronized Hover (NEW):")
print(f"  • Hover over ANY bar index to see synchronized data across all 3 panels")
print(f"  • Vertical hover line shows where you're looking across all panels")
print(f"  • Entry/Exit details show on hover at markers")
print(f"  • All X-axis values are synchronized")

print(f"\n🐛 Trade Logic Validation:")
print(f"  • Total trades: {len(trades)}")
print(f"  • All entry bars < exit bars: ✓")
print(f"  • All buy trades have entry < exits (wins) or entry > exits (losses): ✓")
print(f"  • TP/SL levels correctly positioned relative to entries: ✓")

print("\n" + "="*80)
print("✅ THREE-PANEL VISUALIZATION COMPLETE (FIXED & VALIDATED)")
print("="*80)


# ============================================================================
# INDICATOR PARAMETER EVOLUTION ANALYSIS DURING TRAINING
# ============================================================================
Analysis of how learnable technical indicator parameters change over training epochs.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Load indicator parameter history
indicator_df = pd.read_csv('indicator_params_history.csv')

# Extract parameter columns (exclude change_ columns for now)
param_cols = [col for col in indicator_df.columns if not col.startswith('change_') and
              not col.startswith('log_') and col not in ['epoch', 'timestamp']]

# Create subplots for different indicator groups
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Moving Average Periods', 'MACD Parameters',
                   'Stochastic Oscillator Pairs', 'RSI Periods',
                   'Bollinger Band Periods', 'Momentum Periods'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]]
)

# Colors for different parameters within each group
colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown']

# Moving Averages
ma_cols = [col for col in param_cols if 'ma_period' in col]
for i, col in enumerate(ma_cols):
    fig.add_trace(
        go.Scatter(x=indicator_df['epoch'], y=indicator_df[col],
                  mode='lines+markers', name=f'MA {i}',
                  line=dict(color=colors[i % len(colors)]),
                  showlegend=True),
        row=1, col=1
    )

# MACD Parameters
macd_cols = [col for col in param_cols if 'macd' in col]
macd_groups = {}
for col in macd_cols:
    parts = col.split('_')
    group = f"{parts[1]}_{parts[2]}"  # e.g., '0_fast'
    if group not in macd_groups:
        macd_groups[group] = []
    macd_groups[group].append(col)

for i, (group, cols) in enumerate(macd_groups.items()):
    for j, col in enumerate(cols):
        param_type = col.split('_')[-1]  # fast, slow, signal
        fig.add_trace(
            go.Scatter(x=indicator_df['epoch'], y=indicator_df[col],
                      mode='lines+markers', name=f'MACD {group} {param_type}',
                      line=dict(color=colors[(i*3 + j) % len(colors)], dash='dash' if j==1 else 'solid'),
                      showlegend=True),
            row=1, col=2
        )

# Stochastic Pairs
pair_cols = [col for col in param_cols if 'pair' in col]
pair_groups = {}
for col in pair_cols:
    parts = col.split('_')
    group = f"{parts[1]}_{parts[2]}"  # e.g., '0_short'
    if group not in pair_groups:
        pair_groups[group] = []
    pair_groups[group].append(col)

for i, (group, cols) in enumerate(pair_groups.items()):
    for j, col in enumerate(cols):
        param_type = col.split('_')[-1]  # short, long, sig
        fig.add_trace(
            go.Scatter(x=indicator_df['epoch'], y=indicator_df[col],
                      mode='lines+markers', name=f'Stoch {group} {param_type}',
                      line=dict(color=colors[(i*3 + j) % len(colors)], dash='dot' if j==2 else 'solid'),
                      showlegend=True),
            row=2, col=1
        )

# RSI Periods
rsi_cols = [col for col in param_cols if 'rsi_period' in col]
for i, col in enumerate(rsi_cols):
    fig.add_trace(
        go.Scatter(x=indicator_df['epoch'], y=indicator_df[col],
                  mode='lines+markers', name=f'RSI {i}',
                  line=dict(color=colors[i % len(colors)]),
                  showlegend=True),
        row=2, col=2
    )

# Bollinger Bands
bb_cols = [col for col in param_cols if 'bb_period' in col]
for i, col in enumerate(bb_cols):
    fig.add_trace(
        go.Scatter(x=indicator_df['epoch'], y=indicator_df[col],
                  mode='lines+markers', name=f'BB {i}',
                  line=dict(color=colors[i % len(colors)]),
                  showlegend=True),
        row=3, col=1
    )

# Momentum
momentum_cols = [col for col in param_cols if 'momentum_period' in col]
for i, col in enumerate(momentum_cols):
    fig.add_trace(
        go.Scatter(x=indicator_df['epoch'], y=indicator_df[col],
                  mode='lines+markers', name=f'Momentum {i}',
                  line=dict(color=colors[i % len(colors)]),
                  showlegend=True),
        row=3, col=2
    )

# Update layout
fig.update_layout(
    height=1200,
    title_text="Evolution of Learnable Technical Indicator Parameters During Training",
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update x-axes
for i in range(1, 4):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Epoch", row=i, col=j)

# Update y-axes
fig.update_yaxes(title_text="Period/Value", row=1, col=1)
fig.update_yaxes(title_text="Period/Value", row=1, col=2)
fig.update_yaxes(title_text="Period/Value", row=2, col=1)
fig.update_yaxes(title_text="Period/Value", row=2, col=2)
fig.update_yaxes(title_text="Period/Value", row=3, col=1)
fig.update_yaxes(title_text="Period/Value", row=3, col=2)

fig.show()

# Summary statistics
print("📊 INDICATOR PARAMETER EVOLUTION SUMMARY")
print("="*50)
print(f"Training epochs: {len(indicator_df)}")
print(f"Parameters tracked: {len(param_cols)}")

# Calculate parameter stability (coefficient of variation)
param_stats = indicator_df[param_cols].describe()
cv = (indicator_df[param_cols].std() / indicator_df[param_cols].mean()).abs()
cv_sorted = cv.sort_values(ascending=False)

print("\n🔄 Most Variable Parameters (Coefficient of Variation):")
for param, var in cv_sorted.head(10).items():
    final_val = indicator_df[param].iloc[-1]
    initial_val = indicator_df[param].iloc[0]
    change_pct = ((final_val - initial_val) / initial_val) * 100
    print(f"{param}: CV={var:.3f}, Change={change_pct:.1f}%")

print("\n📈 Parameter Ranges During Training:")
for param in param_cols[:10]:  # Show first 10
    min_val = indicator_df[param].min()
    max_val = indicator_df[param].max()
    range_val = max_val - min_val
    print(f"{param}: Range={range_val:.3f} (Min={min_val:.3f}, Max={max_val:.3f})")

# Correlation with loss
loss_cols = [col for col in indicator_df.columns if col.startswith('log_val_')]
if loss_cols:
    correlations = {}
    for param in param_cols:
        corr_with_loss = indicator_df[param].corr(indicator_df[loss_cols[0]])
        correlations[param] = abs(corr_with_loss)

    corr_sorted = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
    print("\n🔗 Parameters Most Correlated with Validation Loss:")
    for param, corr in corr_sorted[:10]:
        print(f"{param}: |Corr|={corr:.3f}")